In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [2]:
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
print( X_train.shape )
X_train[0][:10, :10]

(60000, 28, 28)


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253],
       [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156]], dtype=uint8)

In [4]:
X_train = X_train / 255
X_test = X_test / 255

In [5]:
X_train.max(), X_train.min()

(1.0, 0.0)

In [6]:
model = tf_keras.models.Sequential([
  tf_keras.layers.Input(shape=(28, 28, 1)),
  tf_keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.MaxPool2D(pool_size=2),
  tf_keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.MaxPool2D(pool_size=2),
  tf_keras.layers.Flatten(),
  tf_keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.Dense(units=10, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 128)               4

In [7]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [8]:
history = model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
750/750 [==============================] - 10s 5ms/step - loss: 0.1367 - accuracy: 0.9570 - val_loss: 0.0745 - val_accuracy: 0.9759
Epoch 2/20
750/750 [==============================] - 3s 4ms/step - loss: 0.0435 - accuracy: 0.9862 - val_loss: 0.0422 - val_accuracy: 0.9876
Epoch 3/20
750/750 [==============================] - 4s 5ms/step - loss: 0.0261 - accuracy: 0.9920 - val_loss: 0.0421 - val_accuracy: 0.9880
Epoch 4/20
750/750 [==============================] - 3s 4ms/step - loss: 0.0188 - accuracy: 0.9937 - val_loss: 0.0408 - val_accuracy: 0.9885
Epoch 5/20
750/750 [==============================] - 4s 5ms/step - loss: 0.0156 - accuracy: 0.9951 - val_loss: 0.0429 - val_accuracy: 0.9884
Epoch 6/20
750/750 [==============================] - 4s 5ms/step - loss: 0.0115 - accuracy: 0.9960 - val_loss: 0.0451 - val_accuracy: 0.9880
Epoch 7/20
750/750 [==============================] - 4s 5ms/step - loss: 0.0114 - accuracy: 0.9961 - val_loss: 0.0463 - val_accuracy: 0.9877
Epoch

In [9]:
model.save("drive/MyDrive/Colab Notebooks/models/mnist-cnn-model.keras")

In [10]:
tf.__version__

'2.15.0'

In [12]:
import pickle
with open("drive/MyDrive/Colab Notebooks/models/mnist-cnn-model2.keras", "wb") as f:
  pickle.dump(model, f)